Join operation có 3 tiêu chí:

* Broadcast joins for unbalanced datasets
* Sort-merge joins for large-scale data
* Multiple joins optimization


Tất cả các phương thức join trên đều được tự động nhận diện tùy vào 2 bảng cần join

Load data vào và build bảng chứa company_detail

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("JSONL_Loader") \
    .getOrCreate()

In [ ]:
# chưa cần chạy vội
hdfs_path = "hdfs://<your_hdfs_namenode>:<port>/path/to/your/file.jsonl"

df_raw = spark.read.json(hdfs_path)
df_raw.printSchema()

In [ ]:
# Thử nghiệm

from pyspark.sql.functions import col

df_flat = df_raw.select(
    col("job_title"),
    col("company_name").alias("main_company_name"), # Đổi tên để tránh trùng lặp
    col("salary"),
    # Trích xuất các trường con từ 'company_detail'
    col("company_detail.company_name").alias("detail_company_name"),
    col("company_detail.declaration"),
    col("company_detail.`Company type`").alias("company_type"), # Dùng dấu `` cho tên cột có khoảng trắng
    col("company_detail.`Company industry`").alias("company_industry"),
    col("company_detail.`Company size`").alias("company_size"),
    col("company_detail.Country"),
    col("company_detail.`Working days`").alias("working_days"),
    col("company_detail.`Overtime policy`").alias("overtime_policy")
    
)

df_flat.printSchema()
df_flat.show(5)

In [ ]:
df_company_detail = df_raw.select(
    
    
    # Trích xuất các trường con từ 'company_detail'
    col("company_detail.company_name").alias("company_key"),
    col("company_detail.declaration"),
    col("company_detail.`Company type`").alias("company_type"), # Dùng dấu `` cho tên cột có khoảng trắng
    col("company_detail.`Company industry`").alias("company_industry"),
    col("company_detail.`Company size`").alias("company_size"),
    col("company_detail.Country"),
    col("company_detail.`Working days`").alias("working_days"),
    col("company_detail.`Overtime policy`").alias("overtime_policy")
    
).distinct()

# df chính để join broadcast
df_job = df_raw.select(
    col("*"),
    col("company_name").alias("company_key_for_join")
).drop("company_detail") # Xóa cột lồng nhau sau khi đã trích xuất Khóa


**Xử lý _broadcast join for unbalanced dataset_** bằng cách sử dụng trường thông tin company_detail chỉ có ở IT_viec rồi join với các bảng chung khác (bởi vì 1 công ty có thể đăng tuyển ở nhiều web).

Cụ thể hơn thì load bảng company_detail này vào bộ nhớ cache của tất cả các worker để làm việc nhanh chóng hơn.

Cách thức hoạt động: Apache Spark sẽ gửi toàn bộ bảng nhỏ hơn đến bộ nhớ đệm (cache) của mỗi node trong cluster. Việc join sau đó diễn ra cục bộ trên mỗi node mà không cần trao đổi dữ liệu qua mạng (shuffle), giúp giảm đáng kể độ trễ.

**Broadcast tự động**

In [ ]:
from pyspark.sql import SparkSession
# ngưỡng 20MB

# cấu hình từ đầu
spark_auto = SparkSession.builder \
    .appName("AutoBroadcastJoin") \
    .config("spark.sql.autoBroadcastJoinThreshold", "20971520") \
    .getOrCreate()
    
# cấu hình trong lúc chạy
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 15728640)  # 15MB

# Spark tự động kiểm tra kích thước df_nho và áp dụng Broadcast Join
df_ket_qua_auto = df_job.join(
    df_company_detail,
    df_job["ma_ky_nang"] == df_company_detail["ma_ky_nang"],
    "inner"
)

# Để xem kế hoạch thực thi (Execution Plan) và xác nhận Broadcast Join được áp dụng:
print("--- Execution Plan (Auto Broadcast) ---")
df_ket_qua_auto.explain()

**Thủ công**

Với dữ liệu của mình thì company_detail chỉ khoảng 300-400KB, còn df trong bộ nhớ ram thì khoảng 1.1 MB - 3.8 MB

In [ ]:
from pyspark.sql.functions import broadcast

df_ket_qua_manual = df_job.join(
    # Sử dụng hàm broadcast() trên DataFrame nhỏ hơn
    broadcast(df_company_detail), 
    df_job["ma_ky_nang"] == df_company_detail["ma_ky_nang"],
    "inner"
)

# Xem kết quả
df_ket_qua_manual.show()

# Xem kế hoạch thực thi để xác nhận (chắc chắn sẽ là BroadcastHashJoin)
print("--- Execution Plan (Manual Broadcast) ---")
df_ket_qua_manual.explain()

**Xử lý _Sort-merge joins for large-scale data_** (nghĩa là xử lý 2 bảng lớn)

Cơ chế: shuffle - sort - merge. Shuffle trên toàn cụm để các key join phù hợp nằm về cùng worker/partition. Sort để sắp xếp các mẫu dữ liệu trong 1 partition 1 cách có trật tự. Merge giống như trong sắp xếp merge-sort, duyệt qua từng mẫu của 2 bên để khớp key join với nhau.

Triển khai:

Đây là tự động nếu gọi join 2 bảng lớn

Vì chưa biết cần join những bảng lớn nào nên cứ partition làm sao cho tối ưu bước lưu trữ. Đến bước này thì shuffle lại. Bước shuffle là bước có cost lớn nhất nên cần tối ưu, nếu bắt buộc (thường dùng) thì partition trước. Nên sử dụng 2 bảng có tính đồng nhất cao để join để bớt bước shuffle

Tải thêm dataset từ huggingface để join
* IT resume https://huggingface.co/datasets/datasetmaster/resumes
* IT Job Roles Skills Dataset - kaggle/huggingface https://huggingface.co/datasets/NxtGenIntern/IT_Job_Roles_Skills_Certifications_Dataset

Triển khai:
* Với IT resume: explode cột skill của 2 bảng job và resume rồi join merge để kiểm tra sự chồng chéo skill giữa 2 bảng, nó sẽ tạo ra một bảng trung gian cực lớn được join theo từng kĩ năng, sau đó có thể ứng dụng tiếp. 
  * Ý nghĩa: 
    * Nhu cầu tương đối: Kỹ năng nào được yêu cầu nhiều (Job_ID lặp lại nhiều) và có nhiều ứng viên (Resume_ID lặp lại nhiều) cùng một lúc. 
    * Mức độ cạnh tranh: Bằng cách GROUP BY Skill_Name và COUNT (Job_ID) / COUNT (Resume_ID), bạn có thể đo lường mức độ cạnh tranh thực tế của một kỹ năng trên thị trường.

Ý tưởng:
* IT job - IT skill theo các key trung gian: 
  * IT role(vị trí công việc): chi tiết hơn về công việc
  * skill name: đề xuất các certificate với mỗi công việc, skill
* IT resume - IT skill:
  * skill/role: đề xuất certificate cần thêm với skill, role ứng tuyển hiện tại

**Xử lý _Multiple Joins Optimization_** 

Mục tiêu là giảm số lần shuffle và điều chỉnh thứ tự join để tối ưu

Tối ưu thứ tự join (Thực hiện phép join giúp giảm kích thước data nhanh nhất trước):
* Join các bảng nhỏ nhất trước: Join bảng thực tế (fact table) với các bảng chiều (dimension tables) nhỏ trước, đặc biệt là các bảng có thể được broadcast. Điều này giúp giảm kích thước của tập kết quả trung gian trước khi join với các bảng lớn hơn.

* Sử dụng CBO (Cost-Based Optimizer): Spark có một bộ tối ưu hóa truy vấn (Query Optimizer) mạnh mẽ. Nó cố gắng tự động tìm ra thứ tự join tốt nhất dựa trên các số liệu thống kê (cardinality, kích thước bảng, v.v.). Trong đây thì cần thu thập thông kê các bảng lớn trước. ANALYZE TABLE ten_bang COMPUTE STATISTICS FOR ALL COLUMNS;

Giảm shuffle:
* Thường xuyên broadcast các bảng nhỏ
* Dùng bucket join

Triển khai với 3 bảng trên:
* Thứ tự join sẽ là job-skill -> resume-skill ->